<a href="https://colab.research.google.com/github/Loris997/Assignments-Loris-Widmer/blob/main/Exam_digital_Organization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **VU Digital Organization final Exam**

Loris Widmer

In [31]:
#I imported all kinds of libraries to be sure:)

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import set_config
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

#Optional imports for Oversampling and LMER-Integration
#from imblearn.over_sampling import SMOTE
set_config(transform_output="pandas")

from google.colab import drive
from google.colab import data_table




In [22]:
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/VU/Holcim_Data.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-22-5a43e7aa3786>:2: DtypeWarning: Columns (0,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/My Drive/VU/Holcim_Data.csv")


In [8]:
data.head()

,na_gl,global_company_key,data_date,data_year_fiscal,fiscal_year_end_month,gck_date,industry_format,level_of_consolidation_company_annual_descriptor,population_source,data_format,...,cik_number,active_inactive_status_marker,current_iso_country_code_incorporation,city,company_legal_name,north_american_industry_classification_code,standard_industry_classification_code,state_province,current_iso_country_code_headquarters,fcf
0,NaN,1023,311287,1987,12.0,1023311287,INDL,C,D,STD,...,2844.0,I,USA,Mountainside,AGS Computers Inc,421430.0,5045.0,NJ,USA,0.027071
1,NaN,1048,311287,1987,12.0,1048311287,INDL,C,D,STD,...,65695.0,I,USA,Houston,ANR Pipeline Co,486210.0,4922.0,TX,USA,NaN
2,NaN,1048,311288,1988,12.0,1048311288,INDL,C,D,STD,...,65695.0,I,USA,Houston,ANR Pipeline Co,486210.0,4922.0,TX,USA,0.184139
3,NaN,1048,311289,1989,12.0,1048311289,INDL,C,D,STD,...,65695.0,I,USA,Houston,ANR Pipeline Co,486210.0,4922.0,TX,USA,0.048957
4,NaN,1048,311290,1990,12.0,1048311290,INDL,C,D,STD,...,65695.0,I,USA,Houston,ANR Pipeline Co,486210.0,4922.0,TX,USA,0.107859


In [9]:
print(data.columns)

Index(['na_gl', 'global_company_key', 'data_date', 'data_year_fiscal',
       'fiscal_year_end_month', 'gck_date', 'industry_format',
       'level_of_consolidation_company_annual_descriptor', 'population_source',
       'data_format', 'ticker_symbol', 'cusip',
       'international_security_identification_number', 'company_name',
       'iso_currency_code', 'assets_total', 'dividends_common_ordinary',
       'dividends_preferred_preference', 'earnings_before_interest_and_taxes',
       'earnings_before_interest', 'employees',
       'operating_activities_net_cash_flow', 'revenue_total',
       'sales_turnover_net', 'stockholders_equity_total',
       'operating_expenses_total', 'research_and_development_expense',
       'selling_general_and_administrative_expense', 'stock_exchange_code',
       'cik_number', 'active_inactive_status_marker',
       'current_iso_country_code_incorporation', 'city', 'company_legal_name',
       'north_american_industry_classification_code',
       'stand

#**Exercise 1**

The following two variables are important:
stockholders_equity_total: This variables shows the total equity held by the companys shareholders. If this is declining it can indicate a decrease in the companys net worth and financial stability. Since the data is from the last 15-20 years this variable can provide insights of the companys capital structure, profitability and ability to generate shareholder value. By examining other variables which contribute to the decline in stockholders equity it is possible to identfy potential areas of concern and suggest strategies.
operating_expenses_total: This variable represetns the total operating expenses by the company. If they are rising it can negatively impact profitability and overall company performance. Analyzing the trend of this variable can help identify cost drivers, efficiency issues, or areas of overspending. If the variable is understood it is possible to identify areas where cost optimization or process improvements can be implemented. Furthermore it is possible to compare the expenses with other companies and provide benchmarks for cost management efficiency.


I will now remove columns and clean the data


In [23]:
# remove columns
data = data.drop(['level_of_consolidation_company_annual_descriptor', 'international_security_identification_number'], axis=1)

Foe example level_of_consolidation_company_annual_descriptor has the same value for all rows, so it doesnt make sense to keep them. Other columns just dont have an impact for the case study.

In [24]:
#replace NA values in operating_activities_net_cash_flow column with 0
data['operating_activities_net_cash_flow'].fillna(0, inplace=True)

In [25]:
#same thing with stockholders_equity_total
data['stockholders_equity_total'].fillna(0, inplace=True)

The NA values have to be replaced to have a consistent cariable typ in the column. By replacing them by 0 it is assumed that no net cash flow was there.

#**Exercise 2**


In [29]:
#take the year 2004
data_2004 = data[data['data_year_fiscal'] == 2004]

#creating the dummy variable
data_2004['industry_dummy'] = (data_2004['industry_format'] == 'Cement').astype(int)

#define columns which are included in the model
columns_to_include = ['earnings_before_interest_and_taxes', 'industry_dummy', 'assets_total', 'employees', 'operating_activities_net_cash_flow', 'revenue_total', 'stockholders_equity_total' ]
data_filtered = data_2004[columns_to_include].dropna()

#linear regression model
X = data_filtered[['industry_dummy', 'assets_total', 'employees', 'operating_activities_net_cash_flow', 'revenue_total', 'stockholders_equity_total']]
y = data_filtered['earnings_before_interest_and_taxes']


X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

#print summary
print(model.summary())

                                    OLS Regression Results                                    
Dep. Variable:     earnings_before_interest_and_taxes   R-squared:                       0.918
Model:                                            OLS   Adj. R-squared:                  0.917
Method:                                 Least Squares   F-statistic:                     2695.
Date:                                Mon, 26 Jun 2023   Prob (F-statistic):               0.00
Time:                                        13:42:11   Log-Likelihood:                -12776.
No. Observations:                                1215   AIC:                         2.556e+04
Df Residuals:                                    1209   BIC:                         2.559e+04
Df Model:                                           5                                         
Covariance Type:                            nonrobust                                         
                                         coef    s

<ipython-input-29-ffd0649c7518>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2004['industry_dummy'] = (data_2004['industry_format'] == 'Cement').astype(int)


I first filter the data to the year 2004. Then I created the dummy variable industry_dummy, with the help of the variable industry_format, setting the value to 1 if it is in the cement industry, 0 if it is not.

Then I selected the relevant columns for the model, including EBIT: 'earnings_before_interest_and_taxes', 'industry_dummy', 'assets_total', 'employees', 'operating_activities_net_cash_flow', 'revenue_total', 'stockholders_equity_total'. I think these variables all have an impact on EBIT and thereofre it makes sense to incluide them into the model.
Alltough I cleaned the data befor, I made sure that the columns work and dropped any columns with a NA(missing values) values.

I then used the statsmodels.api library to perform a linear regression. With the help of add_constant() I added the intercept. With OLS() the model was fitted and fit() calculates the parameters.

Finally I printed the summary of the model.




**Interpretation of the output**

The R-squared value measurees the variance in the dependent variable EBIT. This can be explained by the independent variables from the model. Our R-square value is 0.918, which indicates that 91.8% of the variability in EBIT can be explained by the independant variables.

The adjusted R-squared variable takes the number of independent variables and sample size into account. In our model the value is 0.917 which is very close to the R-squared value which indicates that our model is not overfitting.

Coefficients of independent variables:


*   assets_total: -0.0233
*   employees: -18.4648
- operating_activities_net_cash_flow: 1.0972
- revenue_total: 0.0188
- stockholders_equity_total: 0.1207

The coefficients can be interpretes as followed: if all other variables are held constant, the increase of the variable with one unit will change the EBIT with the callculated value.

The p-value of the variables indicates if the cariable has an statistical impact on the EBIT. In our model assets_total, operating_activities_net_cash_flow and revenue_total have an p-value < 0.05. This indicates thet these variables have an statistical impact. While stockholders_equity_total and industry_dummy have a value of >0.05.


The omnibus and JB test indicate that there is no normal distribution present. This can be the case because of outliers or nonlinearity in the relationship.


# **Exercise 3**


In [30]:
#focus on data from 2000+
data_from_2000 = data[data['data_year_fiscal'] >= 2000]

# Dummy variable
data_from_2000['industry_dummy'] = (data_from_2000['industry_format'] == 'Cement').astype(int)

# columns in the model
columns_to_include = ['earnings_before_interest_and_taxes', 'industry_dummy', 'assets_total', 'employees', 'operating_activities_net_cash_flow', 'revenue_total', 'stockholders_equity_total']
data_filtered = data_from_2000[columns_to_include].dropna()

# Linear regression model
X = data_filtered[['industry_dummy', 'assets_total', 'employees', 'operating_activities_net_cash_flow', 'revenue_total', 'stockholders_equity_total']]
y = data_filtered['earnings_before_interest_and_taxes']


X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())


                                    OLS Regression Results                                    
Dep. Variable:     earnings_before_interest_and_taxes   R-squared:                       0.869
Model:                                            OLS   Adj. R-squared:                  0.869
Method:                                 Least Squares   F-statistic:                 3.315e+04
Date:                                Mon, 26 Jun 2023   Prob (F-statistic):               0.00
Time:                                        14:07:36   Log-Likelihood:            -3.2118e+05
No. Observations:                               25023   AIC:                         6.424e+05
Df Residuals:                                   25017   BIC:                         6.424e+05
Df Model:                                           5                                         
Covariance Type:                            nonrobust                                         
                                         coef    s

<ipython-input-30-28106ab90918>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_from_2000['industry_dummy'] = (data_from_2000['industry_format'] == 'Cement').astype(int)


The model has one main difference: The values are now not only from the year 2004, but from the year 2000 and younger. So the dataset now includes more values and values over time. this kind of model can be more representative since it is a trend analysis. It is possible to make understand the factors better which influence the EBIT over the year.


```
data_from_2000 = data[data['data_year_fiscal'] >= 2000]
```
This is the difference in the code from the model in exercise 3.

After this change we proceeded more or less the same way as in the previous exercise.

**Interpretation of the output**

(I will not again explain what R-square, adjusted R-square... indicate and rather just go into the value of the result(time is running)).

The R-square variable has the value 0.869 while the adjusted R-square is the sam at 0.869. This shows that the included variables dont penalize the goodness fit of the model.

Coefficients of independent variables:


*   assets_total: 0.0745
*   employees: -85.3551
- operating_activities_net_cash_flow: -0.1372
- revenue_total: 0.0378
- stockholders_equity_total: 0.0189

P-values

All independent variables have a p-value < 0.05, which indicates that all variables are statistically significant for EBIT.

The Omnibus and JB test once again indicate nonnormality in the dataset.

**Comparison of the two outputs**

R-square - Adjusted R-square
2004: 0.918 - 0.917
2000  0.869 - 0.869

Both models have a high value which indicates good fits of the models. Nontheless the model of the year 2004 has a bit a higher value which means this model has a slightly better fit.

Coefficients

The main difference is that the industry_dummy variable has a p-value < 0.05 in the second model and therefore is statistically significant. This suggests beeing in the Cement industry has a significant impact on the EBIT over the years.

All other coefficients are more or less the same in both models.



# **Exercise 4**

The coefficient for employees in model from question 2 is -18.4648, in model from question 3 -85.3551.

So if all other variables are constant, a decline of 1 employee manpower would increase the EBIT by 18.4648 (model from question 2) or 85.3551 (model from question 3).

In my logical thinking it should be the other way around, if the employee is increased, the EBIT should be increased as well. It might be that after a certain timespan the coefficient would become positive.

But the result is based on the assumption that the relationship between EBIT and employee is linear and all other factors in the model reamin constant. But the relationship of these two might be influenced by other factors. Additional test and more information would be needed to make a definitive conclusion.

# **Exercise 5**

My general approach to solve this case was a data-driven approach. I used a linear regression model from the statsmodels.api library. The model was used to analyze the relationship between different variables and their input on EBIT  (Earnings before Interest and Taxes).

I started with investigating the dataset provided on OLAT. After a brief look I started to prepare the data for the analyzing. Therefore I deleted some columns which in my opinion were useless and replaced some NAs with 0 values.

After that the data was ready to be used in a linear regression model.
The first step of the model was to filter the data on the wanted timespan (2004 or from 2000 onwards). The I created the dummy variable industry_dummy to see if a comany is in the cement industry or not.
After that I had to decide which columns are relevant to explain the EBIT and include them into the linear regression model. These variables were then used as independent variables (predictors) and EBIT was used as dependent variable.

After that the model was executed and the output was analyzed. To gain insights over the years and include a trend analysis to models were used.

Assumtions for the data-driven approach:


*   The data set is complete, accurate and representative. It reflects the relevant variables and their relationships. An analysis can only be as good as the dataset. If the dataset is super bad, then the output of the model is more or less useless.
*   The relatinship between the independent variables and EBIT has to be linear in order to use a linear regression model.


Limitations:


*   The relationshps which were observed in the analysis do not necessarily indicate a causal relationship between the variables. Additional analysis would be needed.
*   The assumtion of linearity independence was made. If these are violated the reliability of the result might be in danger.
- The selected variables might not capture all factors influencing EBIT. It might be that an importnat variable was missed. It might be that additinal variables would have explained EBIT better.
- A different model could result in different results.

Some of these limitation can be eliminated with further tests.


Three values which could be interesting:

Market conditions: It could be of interest to see what market conditions are present. An index which represents the overal state of the industry would help.

Technological advancements: It could be of interest to see a variable which reflects the technological advancements or innovation in the cement industry. This variable might include things like the adoption of new/more efficient production techniques or  technological disruptions. These might have an impact on the EBIT.

Competition: The competition can have an impact on EBIT. It would help to see the mesure of competition, how many other companies are in the area. This would represent the competitive dynamics of the industry. It might as well influence the pricing of the companies.





